In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
import numpy as np
import os
from pyspark.sql.types import *
import pickle
import tensorflow as tf
from src.util import *
from sklearn.feature_extraction.text import CountVectorizer
# import pyspark.sql.functions as F
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# from pyspark.ml.feature import CountVectorizer, IDF
# from pyspark.sql.functions import udf
# from pyspark.sql import Row
# PUNCTUATION = set(string.punctuation)
# STOPWORDS = set(stopwords.words('english'))



# import tensorflow_hub as hub
# from sklearn import preprocessing
# import codecs
# import string
# import re
# from pyspark.sql.functions import isnan, when, count, col
# import spacy
# from spacy.lang.en import English
# from spacy import displacy
# nlp = spacy.load('en_core_web_md')
# import logging
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


# Load Data

In [2]:
df = spark.read.json(
 's3://aws-logs-816063959671-us-east-1/data/tldr-training-data.jsonl')


In [9]:
subset = df.sample(withReplacement=False, fraction=0.1)
subset.cache()
subset.count()

308706

In [10]:
subset.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- id: string (nullable = true)
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- title: string (nullable = true)



In [35]:
from src import util

In [36]:
# util.word_length('hello world')

2

In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import pyspark
import numpy as np
import string
import re
from pyspark.sql.functions import isnan, when, count, col
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import Row
import pyspark.sql.functions as F
import pickle
PUNCTUATION = set(string.punctuation)
STOPWORDS = set(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
addl_punctuation = set(['...', '`', '¿','⸮', '``', "''"])
PUNCTUATION = PUNCTUATION.union(addl_punctuation)

CONTRACTIONS = {
"ain't": "am not", "aren't": "are not", "can't": "cannot","can't've": "cannot have","'cause": "because",
"could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not",
"doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have",
"he'll": "he will","he's": "he is","how'd": "how did","how'll": "how will",
"how's": "how is","i'd": "i would","i'll": "i will","i'm": "i am","i've": "i have","isn't": "is not",
"it'd": "it would","it'll": "it will","it's": "it is","let's": "let us", "ma'am": "madam", "mayn't": "may not",
"might've": "might have","mightn't": "might not","must've": "must have","mustn't": "must not",
"needn't": "need not","oughtn't": "ought not","shan't": "shall not","sha'n't": "shall not","she'd": "she would",
"she'll": "she will","she's": "she is","should've": "should have","shouldn't": "should not","that'd": "that would",
"that's": "that is","there'd": "there had","there's": "there is","they'd": "they would","they'll": "they will",
"they're": "they are","they've": "they have","wasn't": "was not","we'd": "we would","we'll": "we will",
"we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what're": "what are",
"what's": "what is","what've": "what have","where'd": "where did","where's": "where is","who'll": "who will",
"who's": "who is","won't": "will not","wouldn't": "would not","you'd": "you would","you'll": "you will",
"you're": "you are"
}

def clean_text(text, remove_stopwords=True):
    text = text.lower()

    if True:
        text = text.split()
        new_text = [CONTRACTIONS[w] if w in CONTRACTIONS else w for w in text]

        text = " ".join(new_text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in PUNCTUATION]
    if remove_stopwords==True:
        tokens = [w for w in tokens if w not in STOPWORDS]
    return ' '.join(tokens)

def word_length(string):
    tokens = word_tokenize(string)
    tokens = [w for w in tokens if w not in PUNCTUATION]
    return len(tokens)

def clean_data(df, n_words_summary=50, remove_stopwords=True):
    # Get rid of all rows where subreddit is null (these are spam)
    df = df.filter(df.subreddit.isNotNull())
    # Lowercase columns:
    for col in ['body','content','normalizedBody','subreddit','summary','title']:
        df = df.withColumn(col, F.lower(F.col(col)))
    # Converts 'null' strings in the title column back to null values
    df = df.withColumn('title', when(df.title == 'null', F.lit(None)).otherwise(df.title))
    

    # Creat edit(bool) and edit_len columns, while removing 'edit:%' from summary column
    split_col = F.split(df['summary'], '(edit:|[^a-z]edit)')
    df = df.withColumn('edit', split_col.getItem(1))
    df = df.withColumn('summary', split_col.getItem(0))
    function = udf(word_length, LongType())
    df = df.withColumn('summary_len', function(df.summary))
        # Creates edit_len column, number of words from 'edit'
    df = df.withColumn('edit', df.edit).na.fill('')
    df = df.withColumn('edit_len', function(df.edit))
        # Converts -1 in edit_len column to null
    df = df.withColumn('edit_len',
        when(df.edit_len == -1, F.lit(0)).otherwise(df.edit_len))
    df = df.withColumn('edit', when(df.edit.isNull(), F.lit(0)).otherwise(1))
    # Remove all rows where summary contains less than 5 words
    df = df.filter(df.summary_len >= 5)
    # Remove all rows where summary contains greater than n_words_summary words
    df = df.filter((df.summary_len <= n_words_summary))
    # Remove all rows where the summary length is not less than 50% of the content length
    df = df.filter(df.summary_len <= df.content_len*0.5)
    # Clean Content column
    cleantext_udf = udf(clean_text, StringType())
    df = df.withColumn('content', cleantext_udf(df.content, F.lit(remove_stopwords)))
    df = df.withColumn('summary', cleantext_udf(df.summary, F.lit(False)))
    return df

In [79]:
newdf = clean_data(subset)

In [80]:
newdf.count()

267253

In [119]:
pdf = newdf.toPandas()

In [120]:
pdf.drop(['body', 'normalizedBody', 'author', 'id', 'subreddit', 'subreddit_id',
         'title'], axis=1, inplace=True)

In [121]:
pdf.head()

,content,content_len,summary,summary_len,edit,edit_len
0,10 month old son mcdonalds playplace little go...,142,my 10 month old molests a 5 year old,9,1,0
1,great image 100 certain agree easy say `` rebe...,184,do not blame miserable parents for not realizi...,22,1,0
2,meant files mostly word really well actually g...,181,i act/post like i am high or something late at...,11,1,0
3,hit defensive timings know defensive timings w...,121,either you invest in queens for aa and oversee...,27,1,0
4,kinda issue putting blame one party yes agree ...,231,riot should have had booths but at the same ti...,15,1,0


In [123]:
pdf = pdf.replace('', np.NaN)
pdf = pdf.replace(float('nan'), np.NaN)
pdf.dropna(inplace=True)

In [124]:
# Inspecting some of the reviews
for i in range(15):
    print("Review #",i+1)
    print(pdf.summary[i])
    print(pdf.content[i])
    print()

Review # 1
my 10 month old molests a 5 year old
10 month old son mcdonalds playplace little goofball adorable little girl climbs well cutest little thing ever 5. walks fiancee asks `` play '' points son said sure climbs starts playing well twenty minutes giggling laughing suddenly screams bloody murder turn around started laughing turns 10 month old son big 5 year old pinned pulling dress trying bite cheek adorably creepy climb pull easier said done ran promptly left mom

Review # 2
do not blame miserable parents for not realizing they would be miserable blame the culture that told them they would not be
great image 100 certain agree easy say `` rebelled '' often facts matter grow environment full certain messages likely end believing true go college want life kids get 9-5 job responsible etc fair blame people listening things told entire lives insight realize bullshit escape rather need start challenging messages get anything smug sense superiority blaming people saying `` oh miserabl

# Create Vocabulary

In [169]:
cv = CountVectorizer(strip_accents=ascii)
cv.fit_transform(pdf['content'])
vocab = cv.vocabulary_

In [170]:
len(vocab)

302392